# 排球少年 Break TCG 卡片資料處理工具

此筆記本用於處理排球少年 Break TCG 的卡片資料 CSV 檔案。

## 主要功能
1. 從「完整技能」欄位中提取「時機點」資訊
2. 清理時機點格式（移除符號如 `[`, `]`, `=`, `←` 等）
3. 將卡片分類為「角色卡」和「事件卡」
4. 輸出為兩個獨立的 CSV 檔案

## 1. 匯入必要的套件

In [1]:
import csv
import re
import io

## 2. 定義目標欄位結構

In [2]:
# 設定欄位順序 (Target Schema)
FIELDNAMES = [
    '卡片類型', '卡片編號', '卡片名稱', '時機點', '稀有度', '位置', 
    '發球', '攔網', '接球', '托球', '攻擊', '完整技能', '注釋'
]

## 3. 時機點提取與清理函數

從完整技能文字中提取時機點資訊，並清理格式。

In [3]:
def clean_timing(text):
    """
    從完整技能中提取時機點，格式為 [時機][時機2]
    回傳: (提取出的時機點字串, 剩餘的技能描述)
    """
    if not text or text == '-':
        return '-', '-'
    
    # 匹配開頭連續的 [...] 區塊
    # Regex 解釋: ^(\[[^\]]+\]\s*)+ 匹配開頭的一個或多個 [xxx]
    match = re.match(r'^((?:\[[^\]]+\]\s*)+)', text)
    
    if match:
        full_timing_block = match.group(1)
        remaining_skill = text[len(full_timing_block):].strip()
        
        # 處理時機點字串: 移除 [ ] = ← 等符號，改用逗號分隔
        # 1. 移除 [, ]
        # 2. 移除 = (例如 [=登場] -> 登場)
        # 3. 移除 ← (例如 [←攻擊區] -> 攻擊區)
        timings = re.findall(r'\[(.*?)\]', full_timing_block)
        cleaned_timings = []
        for t in timings:
            # 清理內部符號
            t_clean = t.replace('=', '').replace('←', '').strip()
            if t_clean:
                cleaned_timings.append(t_clean)
        
        timing_str = ','.join(cleaned_timings)
        return timing_str, remaining_skill
    else:
        return '-', text

## 4. 主要處理函數

讀取輸入檔案，處理資料，並輸出為兩個獨立的 CSV 檔案。

In [4]:
def process_file(input_file, char_output, event_output):
    characters = []
    events = []

    with open(input_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        
        for row in reader:
            # 處理每一行
            original_skill = row.get('完整技能', '-')
            timing, skill_content = clean_timing(original_skill)
            
            new_row = {
                '卡片類型': row.get('卡片類型', '-'),
                '卡片編號': row.get('卡片編號', '-'),
                '卡片名稱': row.get('卡片名稱', '-'),
                '時機點': timing,
                '稀有度': row.get('稀有度', '-'),
                '位置': row.get('位置', '-'), # 事件卡此欄位通常為 -
                '發球': row.get('發球', '-'),
                '攔網': row.get('攔網', '-'),
                '接球': row.get('接球', '-'),
                '托球': row.get('托球', '-'),
                '攻擊': row.get('攻擊', '-'),
                '完整技能': skill_content,
                '注釋': row.get('注釋', '-')
            }
            
            # 分類
            c_type = new_row['卡片類型'].upper().strip()
            if c_type == 'CHARACTER':
                characters.append(new_row)
            elif c_type == 'EVENT':
                # 事件卡確保位置是 -
                if new_row['位置'] == '': new_row['位置'] = '-'
                events.append(new_row)

    # 寫入檔案
    with open(char_output, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=FIELDNAMES)
        writer.writeheader()
        writer.writerows(characters)

    with open(event_output, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=FIELDNAMES)
        writer.writeheader()
        writer.writerows(events)
    
    print(f"✅ 處理完成！")
    print(f"   角色卡數量: {len(characters)}")
    print(f"   事件卡數量: {len(events)}")
    print(f"   角色卡輸出: {char_output}")
    print(f"   事件卡輸出: {event_output}")

## 5. 使用範例

修改以下路徑以符合您的檔案位置，然後執行此 Cell。

In [5]:
# 使用範例
input_file = 'public/pool/梟谷/排球少年Break TCG卡池（共編檔案） - 梟谷.csv'
char_output = '梟谷卡表 - 梟谷角色卡.csv'
event_output = '梟谷卡表 - 梟谷事件卡.csv'

process_file(input_file, char_output, event_output)

FileNotFoundError: [Errno 2] No such file or directory: 'public/pool/梟谷/排球少年Break TCG卡池（共編檔案） - 梟谷.csv'

## 6. 測試時機點提取功能

可以在此測試 `clean_timing` 函數的效果。

In [ ]:
# 測試範例
test_cases = [
    '[=登場] 這是技能描述',
    '[←攻擊區][回合開始] 技能內容',
    '[時機1][時機2][時機3] 完整的技能說明',
    '沒有時機點的技能',
    '-'
]

print("時機點提取測試：\n")
for test in test_cases:
    timing, skill = clean_timing(test)
    print(f"原始: {test}")
    print(f"時機點: {timing}")
    print(f"技能: {skill}")
    print("-" * 50)